In [1]:
# imports
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import StratifiedKFold, train_test_split
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler # To preprocess data for SVM - greatly improves performance
from sklearn.svm import SVC
from pytorch_tabr import TabRClassifier as TabRClassifier_
from itertools import product
from typing import Tuple, Union, Optional, Callable
from tqdm import tqdm

In [2]:
def SVMClassifier(**hyperparams):
    return make_pipeline(StandardScaler(), SVC(**hyperparams))

In [3]:
class TabRClassifier(TabRClassifier_):
    def __init__(self, **kwargs):
        # selection_function_name="sparsemax",
        # context_dropout=0.5,
        # context_sample_size=2000,
        # num_embeddings={"type": "PLREmbeddings", "n_frequencies": 32, "frequency_scale": 32, "d_embedding": 32, "lite": False},
        super().__init__(**kwargs)
        self.type_embeddings="one-hot"
        self.device_name="cpu"
        self.optimizer_params={"lr": 2e-4}
        self.d_main=48
        self.d_multiplier = 1.5
        self.context_dropout = 0
        self.dropout0 = 0
        self.dropout1 = 0
        self.context_size = 12
        
    def fit(
        self,
        X_train: pd.DataFrame,
        y_train: pd.DataFrame,
        **kwargs
    ) -> None:
        super().fit(X_train=X_train.values, y_train=y_train.values,
                    max_epochs=1, batch_size=25, **kwargs)
        
    def predict(self, X: pd.DataFrame):
        return super().predict(X=X.values)
        

In [4]:
# constants
SEED = 42
NUM_SPLITS = 10
TARGET = "decision"
MODELS = [XGBClassifier, SVMClassifier, TabRClassifier]

# XGBCLASSIFIER_HYPERPARAMETERS = {
#     "max_depth": [3, 4, 5, 6, 7, 8],
#     "min_child_weight": [1, 4, 8, 16],
# }

XGBCLASSIFIER_HYPERPARAMETERS = {
    "max_depth": [3, 4],
    "min_child_weight": [1, 4],
}

SVMCLASSIFIER_HYPERPARAMETERS = {
    "degree": [2, 3, 4, 5]
}

TABRCLASSIFIER_HYPERPARAMETERS = {
   "d_main": [24, 48],
    "d_multiplier": [1.2, 1.5]
}

HYPERPARAMETERS = {
    XGBClassifier.__name__ : XGBCLASSIFIER_HYPERPARAMETERS,
    SVMClassifier.__name__ : SVMCLASSIFIER_HYPERPARAMETERS,
    TabRClassifier.__name__ : TABRCLASSIFIER_HYPERPARAMETERS
}

In [5]:
def one_hot_encode(df, features):
    for feature in features:
        dummies = pd.get_dummies(df.loc[:, feature], prefix=feature)
        df = pd.concat([df, dummies], axis=1)
        df = df.drop(feature, axis=1)
    return df

In [6]:
# load dataset
dataset = pd.read_csv("SpeedDating.csv", index_col=0)

# remove redundant columns
subset = ['gender', 'age', 'age_o', 'race', 'race_o', 'importance_same_race', 'importance_same_religion',
          'pref_o_attractive', 'pref_o_sincere', 'pref_o_intelligence',
          'pref_o_funny', 'pref_o_ambitious', 'pref_o_shared_interests', 'attractive_o', 'sinsere_o', 'intelligence_o', 'funny_o',
          'ambitous_o', 'shared_interests_o', 'attractive_important', 'sincere_important', 'intellicence_important', 'funny_important', 'ambtition_important',
          'shared_interests_important', 'attractive', 'sincere', 'intelligence', 'funny', 'ambition', 'attractive_partner', 'sincere_partner',
          'intelligence_partner', 'funny_partner', 'ambition_partner', 'shared_interests_partner',
          'sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts',
          'music', 'shopping', 'yoga',
          'interests_correlate', 'expected_happy_with_sd_people', 'expected_num_matches', 'expected_num_interested_in_me',
          'like', 'guess_prob_liked', 'decision']

dataset = dataset.loc[:, subset]
dataset.loc[:, 'gender'] = (dataset.loc[:, 'gender'] == 'female') # one hot encode gender
dataset = one_hot_encode(dataset, ['race', 'race_o'])
dataset = dataset.apply(pd.to_numeric, errors='coerce', axis=1)
dataset = dataset.fillna(dataset.mean())
print(dataset.head())
X, y = dataset.loc[:, dataset.columns != TARGET], dataset.loc[:, TARGET]
BASELINE = np.sum(y == 1) / np.sum(y == 0)


C:\Users\mracz\AppData\Local\Temp\ipykernel_22340\3974884003.py:2: DtypeWarning: Columns (4,11,12,16,17,18,19,20,40,41,42,43,44,45,52,53,54,55,56,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,108,110) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv("SpeedDating.csv", index_col=0)


    gender   age  age_o  importance_same_race  importance_same_religion  \
id                                                                        
1      1.0  21.0   27.0                   2.0                       4.0   
2      1.0  21.0   22.0                   2.0                       4.0   
3      1.0  21.0   22.0                   2.0                       4.0   
4      1.0  21.0   23.0                   2.0                       4.0   
5      1.0  21.0   24.0                   2.0                       4.0   

    pref_o_attractive  pref_o_sincere  pref_o_intelligence  pref_o_funny  \
id                                                                         
1                35.0            20.0                 20.0          20.0   
2                60.0             0.0                  0.0          40.0   
3                19.0            18.0                 19.0          18.0   
4                30.0             5.0                 15.0          40.0   
5                3

In [7]:
def train_test_models(models, hyperparameters, X, y):
    accuracies = {}
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=SEED)
    for model_class in models:
        model_hyperparameters = hyperparameters[model_class.__name__]
        accuracies[model_class.__name__] = []
        for hyperparams in product(*model_hyperparameters.values()):
            kwargs = dict(zip(model_hyperparameters.keys(), hyperparams))
            model = model_class(**kwargs)
            model.fit(X_train, y_train)
            acc = np.mean(model.predict(X_test) == np.array(y_test))
            accuracies[model_class.__name__].append((acc, kwargs))
    return accuracies

In [8]:
def cross_validate_models(models, kwargs_lists, X, y):
    accuracies = {}
    kf = StratifiedKFold(n_splits=NUM_SPLITS, shuffle=True, random_state=SEED)
    for model_class in models:
        accuracies[model_class.__name__] = []
        for kwargs in kwargs_lists[model_class.__name__]:
            s=0
            model = model_class(**kwargs)
            for train_idx, test_idx in kf.split(X, y):
                X_train, y_train = X.iloc[train_idx, :], y.iloc[train_idx]
                X_test, y_test = X.iloc[test_idx, :], y.iloc[test_idx]
                model.fit(X_train, y_train)
                s += np.mean(model.predict(X_test) == np.array(y_test))
            accuracies[model_class.__name__].append((s/NUM_SPLITS, kwargs))
    return accuracies

In [15]:
def get_top_kwargs(models, accuracies, cutoff):
    top_kwargs = {}
    for model_class in models:
        _, top_models_ = zip(*(sorted(accuracies[model_class.__name__], key=lambda x: x[0])[-int(cutoff*len(accuracies[model_class.__name__])):]))
        top_kwargs[model_class.__name__] = list(top_models_)
    return top_kwargs

In [16]:
def get_rashomon_sets(models, hyperparameters, X, y, initial_cutoff=0.1, top=0.01):
    assert top <= initial_cutoff
    initial_accuracies = train_test_models(models, hyperparameters, X, y)
    filtered_kwargs_lists = get_top_kwargs(models, initial_accuracies, initial_cutoff)
    final_accuracies = cross_validate_models(models, filtered_kwargs_lists, X, y)
    top_kwargs = get_top_kwargs(models, final_accuracies, top/initial_cutoff)
    return top_kwargs
    

In [17]:
rashomon_sets = get_rashomon_sets(MODELS, HYPERPARAMETERS, X, y, 0.75, 0.5)

C:\Users\mracz\Documents\ML\XAI-team-project\pytorch_tabr\base_model.py:258: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


 epochs:   0%|          | 0/1 [00:00<?, ?it/s]

 batches:   0%|          | 0/301 [00:00<?, ?it/s]

 epochs:   0%|          | 0/1 [00:00<?, ?it/s]

 batches:   0%|          | 0/301 [00:00<?, ?it/s]

...

 epochs:   0%|          | 0/1 [00:00<?, ?it/s]

 batches:   0%|          | 0/301 [00:00<?, ?it/s]

 epochs:   0%|          | 0/1 [00:00<?, ?it/s]

 batches:   0%|          | 0/301 [00:00<?, ?it/s]

2nd TEST:
{'XGBClassifier': [{'max_depth': 3, 'min_child_weight': 4}, {'max_depth': 4, 'min_child_weight': 4}], 'SVMClassifier': [{'degree': 4}, {'degree': 5}], 'TabRClassifier': [{'d_main': 48, 'd_multiplier': 1.2}, {'d_main': 48, 'd_multiplier': 1.5}]}


In [ ]:
# Train and test the model
# kf = StratifiedKFold(n_splits=NUM_SPLITS, shuffle=True, random_state=SEED)
# svm_classifier = make_pipeline(StandardScaler(), SVC())
# s=0
# for train_idx, test_idx in kf.split(X, y):
#     X_train, y_train = X.iloc[train_idx, :], y.iloc[train_idx]
#     X_test, y_test = X.iloc[test_idx, :], y.iloc[test_idx]
#     svm_classifier.fit(X_train, y_train)
#     print('Split accuracy: ', np.mean(svm_classifier.predict(X_test) == np.array(y_test)))
#     s += np.mean(svm_classifier.predict(X_test) == np.array(y_test))
#     print('Accuracy for class 1 [person wanted to match]', np.sum( np.logical_and(svm_classifier.predict(X_test) == 1, svm_classifier.predict(X_test) == y_test))/np.sum(y_test))
# print('Average accuracy:')
# print(s/NUM_SPLITS)

In [ ]:
# Train and test the model
# kf = StratifiedKFold(n_splits=NUM_SPLITS, shuffle=True, random_state=SEED)
# xgboost = XGBClassifier(max_depth=4)
# s=0
# for train_idx, test_idx in kf.split(X, y):
#     X_train, y_train = X.iloc[train_idx, :], y.iloc[train_idx]
#     X_test, y_test = X.iloc[test_idx, :], y.iloc[test_idx]
#     xgboost.fit(X_train, y_train)
#     print('Split accuracy: ', np.mean(xgboost.predict(X_test) == np.array(y_test)))
#     s += np.mean(xgboost.predict(X_test) == np.array(y_test))
#     print('Accuracy for class 1 [person wanted to match]', np.sum( np.logical_and(xgboost.predict(X_test) == 1, xgboost.predict(X_test) == y_test))/np.sum(y_test))
# print('Average accuracy:')
# print(s/NUM_SPLITS)